In [58]:
import numpy as np
import pandas as pd

# loading the data from local file
data = pd.read_csv('dataset8.csv')
data.head()

,bpm,pace,time,temperature,gap,dist_quater,count_quater,steps_quater,puls_quater,weight_yeqr
0,149,357,4305,20,2,372590,27,1160574,60,77.1
1,147,363,4403,21,4,372480,27,1172636,60,77.1
2,149,366,4468,24,3,372320,27,1184919,60,77.1
3,153,374,5205,29,2,381430,27,1189566,60,77.2
4,151,356,6564,22,3,363030,26,1181979,60,77.2


In [59]:
print(data.shape)
print(data.isna().sum().sum())

(270, 10)
0


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   bpm           270 non-null    int64  
 1   pace          270 non-null    int64  
 2   time          270 non-null    int64  
 3   temperature   270 non-null    int64  
 4   gap           270 non-null    int64  
 5   dist_quater   270 non-null    int64  
 6   count_quater  270 non-null    int64  
 7   steps_quater  270 non-null    int64  
 8   puls_quater   270 non-null    int64  
 9   weight_yeqr   270 non-null    float64
dtypes: float64(1), int64(9)
memory usage: 21.2 KB


In [61]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('bpm', axis=1),
    data.bpm,
    test_size=0.15,
    random_state=42
)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [62]:
X_train.mean(axis = 0)

array([0.28722123, 0.28163802, 0.5748216 , 0.03582482, 0.5601904 ,
       0.60779557, 0.64834798, 0.35633188, 0.50295973])

In [63]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

cb = CatBoostRegressor(
    random_seed=42,
)

# Сетка параметров
param_grid = {
    'iterations': [200],
    'learning_rate': [0.1],
    'depth': [5],
}

# GridSearchCV
model = GridSearchCV(
    estimator=cb,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=4,         # 4-кратная кросс-валидация
    n_jobs=-1,    # параллельно на всех ядрах
    verbose=2
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

print("Best params:", model.best_params_)
print("Best RMSE:", -model.best_score_)
print('-----')
print("Test RMSE = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Train RMSE = %.4f" % np.sqrt(mean_squared_error(y_train, y_train_pred)))

Fitting 4 folds for each of 1 candidates, totalling 4 fits
0:	learn: 8.8656502	total: 1.13ms	remaining: 225ms
1:	learn: 8.4652856	total: 1.81ms	remaining: 179ms
2:	learn: 8.0101628	total: 2.34ms	remaining: 153ms
3:	learn: 7.5939475	total: 2.91ms	remaining: 142ms
4:	learn: 7.2404075	total: 3.43ms	remaining: 134ms
5:	learn: 6.8849980	total: 3.97ms	remaining: 128ms
6:	learn: 6.5765056	total: 4.51ms	remaining: 124ms
7:	learn: 6.3045269	total: 5.1ms	remaining: 122ms
8:	learn: 6.0461950	total: 5.97ms	remaining: 127ms
9:	learn: 5.8022307	total: 6.67ms	remaining: 127ms
10:	learn: 5.6119640	total: 7.34ms	remaining: 126ms
11:	learn: 5.3994655	total: 7.99ms	remaining: 125ms
12:	learn: 5.2108323	total: 8.54ms	remaining: 123ms
13:	learn: 5.0674689	total: 9.12ms	remaining: 121ms
14:	learn: 4.9208185	total: 9.65ms	remaining: 119ms
15:	learn: 4.7861690	total: 10.2ms	remaining: 117ms
16:	learn: 4.6606068	total: 10.7ms	remaining: 115ms
17:	learn: 4.5247731	total: 11.3ms	remaining: 114ms
18:	learn: 4.421

In [64]:
best_rf = model.best_estimator_
importances = best_rf.feature_importances_

for i, val in enumerate(importances):
    print(f"Feature {i}: {val*10:.0f}")

Feature 0: 404
Feature 1: 46
Feature 2: 47
Feature 3: 34
Feature 4: 53
Feature 5: 52
Feature 6: 70
Feature 7: 189
Feature 8: 104
